In [4]:
import inseq

from t5_lit_module import LitT5

# Load model from checkpoint
model = LitT5.load_from_checkpoint(
    checkpoint_path="checkpoints/model.ckpt",
)


/Users/barberademol/Desktop/ik-nlp-tutorials/venv/lib/python3.8/site-packages/torchmetrics/text/bert.py:179: UserWarning: The argument `model_name_or_path` was not specified while it is required when the default `transformers` model is used. It will use the default recommended model - 'roberta-large'.
  warn(


# See how output is generated

In [6]:
inseq_model = inseq.load_model(model.model, "saliency")
inseq_model.attribute(
    "premise: Taller runner in orange and smaller one in blue run side by side around \
    track in crowded stadium. hypothesis: Two people are running together in the stadium."
).show()


/Users/barberademol/Desktop/ik-nlp-tutorials/venv/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Attributing with saliency...: 100%|██████████| 21/21 [00:14<00:00,  1.41it/s]


,▁Two,▁people,▁are,▁running,▁together,▁in,▁the,▁stadium,▁is,▁,a,▁,re,phra,s,ing,▁of,▁Tall,er,</s>
▁,0.036,0.03,0.018,0.024,0.014,0.014,0.015,0.051,0.02,0.032,0.026,0.031,0.032,0.043,0.035,0.027,0.033,0.05,0.073,0.037
premise,0.011,0.012,0.014,0.011,0.008,0.011,0.011,0.012,0.018,0.016,0.033,0.019,0.02,0.047,0.039,0.024,0.032,0.016,0.011,0.014
:,0.027,0.023,0.022,0.022,0.014,0.016,0.018,0.016,0.019,0.024,0.029,0.027,0.019,0.033,0.027,0.024,0.029,0.043,0.031,0.027
▁Tall,0.109,0.06,0.033,0.07,0.013,0.019,0.015,0.01,0.028,0.065,0.037,0.091,0.029,0.04,0.049,0.037,0.05,0.163,0.134,0.048
er,0.107,0.069,0.028,0.048,0.013,0.024,0.014,0.009,0.022,0.083,0.039,0.081,0.034,0.048,0.042,0.028,0.04,0.151,0.273,0.071
▁,0.034,0.047,0.02,0.03,0.016,0.019,0.017,0.022,0.02,0.029,0.025,0.049,0.029,0.049,0.029,0.03,0.028,0.045,0.046,0.06
runner,0.036,0.066,0.027,0.046,0.027,0.037,0.028,0.019,0.029,0.038,0.067,0.055,0.064,0.094,0.045,0.042,0.048,0.04,0.06,0.083
▁in,0.02,0.017,0.013,0.018,0.018,0.017,0.015,0.013,0.013,0.018,0.014,0.014,0.015,0.016,0.013,0.018,0.02,0.018,0.019,0.037
▁orange,0.014,0.01,0.008,0.01,0.016,0.012,0.009,0.009,0.008,0.021,0.011,0.011,0.007,0.015,0.011,0.018,0.019,0.012,0.012,0.038
▁and,0.025,0.024,0.021,0.021,0.04,0.028,0.017,0.015,0.018,0.037,0.014,0.02,0.014,0.022,0.02,0.039,0.038,0.025,0.025,0.071


# Directly comparing two forced outputs

Note: Outputs must have the same length. 

In this example, you can see that "the two people" is chosen instead of "not two people" mainly due to the word "together", which makes sense.

For example, we can use PairAggregator to compare the feature importance of two different models that perform the explanation task to identify the strengths and weaknesses of each model and potentially improve their performance.

In [17]:
from inseq.data.aggregator import AggregatorPipeline, ContiguousSpanAggregator, SequenceAttributionAggregator, PairAggregator

# Perform the attribution with forced decoding. Return convergence deltas, probabilities and target attributions.
out = inseq_model.attribute(
    [
        "premise: Taller runner in orange and smaller one in blue run side by side around track in crowded stadium. hypothesis: Two people are running together in the stadium.",
        "premise: Taller runner in orange and smaller one in blue run side by side around track in crowded stadium. hypothesis: Two people are running together in the stadium.",
    ],
    [
        "Taller runner and smaller one in blue are the two people.",
        "Taller runner and smaller one in blue are not two people.",
    ],
    attribute_target=True,
    step_scores=["probability"],
    include_eos_baseline=False,
)

# Aggregation pipeline composed by two steps:
# 1. Aggregate contiguous tokens across all attribution dimensions
# 2. Aggregate the last dimension of the neuron-level attribution to make it token-level
squeezesum = AggregatorPipeline([ContiguousSpanAggregator, SequenceAttributionAggregator])

# aggregrate
actual_output = out.sequence_attributions[0].aggregate(aggregator=squeezesum, target_spans=(1, 3))
forced_output = out.sequence_attributions[1].aggregate(aggregator=squeezesum, target_spans=(1, 3))

# Take the diff of the scores of the two attribution and show it
actual_output.show(aggregator=PairAggregator, paired_attr=forced_output)


Attributing with saliency...: 100%|██████████| 16/16 [00:24<00:00,  1.65s/it]


,▁Tall,er▁,runner,▁and,▁smaller,▁one,▁in,▁blue,▁are,▁the → ▁not,▁two,▁people,.,</s>
▁,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002,-0.005,-0.003,0.004,0.001
premise,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,-0.0,-0.0,0.001,-0.0
:,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,-0.003,-0.0,0.002,-0.0
▁Tall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.008,-0.004,-0.001,-0.001,-0.001
er,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002,-0.002,-0.006,-0.001,-0.0
▁,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.005,0.002,0.001,-0.0
runner,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,-0.005,0.007,-0.001,-0.001
▁in,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003,-0.01,-0.009,0.006,0.0
▁orange,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006,-0.013,-0.01,0.005,-0.0
▁and,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004,-0.031,-0.028,0.019,-0.0


# Compare actual output with another possible output

Why our model generated its output sequence rather than another one that we consider to be more likely. Inseq uses a feature attribution algorithm to identify the most important features for predicting the original label and the contrastive label. The difference in feature importance between the original and contrastive examples is then used to attribute importance to the features for predicting the original label. By using custom attribution targets in this process, the feature importance can be attributed specifically to the features that are relevant for predicting the target labels of interest. This can provide more targeted and meaningful insights into the model's decision-making process.

In [18]:
from inseq.attr.step_functions import probability_fn

# Simplified implementation of inseq.attr.step_functions.contrast_prob_diff_fn
# Works only for encoder-decoder models!
def example_prob_diff_fn(
    # Default arguments in attribution_model.forward
    attribution_model,
    forward_output,
    encoder_input_embeds,
    encoder_attention_mask,
    decoder_input_ids,
    decoder_attention_mask,
    target_ids,
    # Extra arguments for our use case
    contrast_ids,
    contrast_attention_mask,
    # We use kwargs to collect unused default arguments
    **kwargs,
):
    """Custom attribution function returning the difference between next step probability for
    candidate generation vs. a contrastive alternative, answering the question "Which features
    were salient in deciding to pick the selected token rather than its contrastive alternative?"

    Extra args:
        contrast_ids: Tensor containing the ids of the contrastive input to be compared to the
            regular one.
        contrast_attention_mask: Tensor containing the attention mask for the contrastive input
    """
    # We truncate contrastive ids and their attention map to the current generation step
    contrast_decoder_input_ids = contrast_ids[:, : decoder_input_ids.shape[1]].to(attribution_model.device)
    contrast_decoder_attention_mask = contrast_attention_mask[:, : decoder_attention_mask.shape[1]].to(
        attribution_model.device
    )
    # We select the next contrastive token as target
    contrast_target_ids = contrast_ids[:, decoder_input_ids.shape[1]].to(attribution_model.device)
    # Forward pass with the same model used for the main generation, but using contrastive inputs instead
    contrast_output = attribution_model.model(
        inputs_embeds=encoder_input_embeds,
        attention_mask=encoder_attention_mask,
        decoder_input_ids=contrast_decoder_input_ids,
        decoder_attention_mask=contrast_decoder_attention_mask,
    )
    # Return the prob difference as target for attribution
    model_probs = probability_fn(attribution_model, forward_output, target_ids)
    contrast_probs = probability_fn(attribution_model, contrast_output, contrast_target_ids)
    return model_probs - contrast_probs


In [19]:
# Register the function defined above
# Since outputs are still probabilities, contiguous tokens can still be aggregated using product
inseq.register_step_function(
    fn=example_prob_diff_fn,
    identifier="example_prob_diff",
    aggregate_map={"span_aggregate": lambda x: x.prod(dim=1, keepdim=True)},
)

# Pre-compute ids and attention map for the contrastive target
contrast = inseq_model.encode("Taller runner and smaller one in blue are not two people.", as_targets=True)

# Perform the contrastive attribution:
# Regular (forced) target -> "Non posso crederci."
# Contrastive target      -> "Non posso crederlo."
# contrast_ids & contrast_attention_mask are kwargs defined in the function definition
out = inseq_model.attribute(
    "premise: Taller runner in orange and smaller one in blue run side by side around track in crowded stadium. hypothesis: Two people are running together in the stadium.",
    "Taller runner and smaller one in blue are the two people.",
    attributed_fn="example_prob_diff",
    contrast_ids=contrast.input_ids,
    contrast_attention_mask=contrast.attention_mask,
    attribute_target=True,
    # We also visualize the step score
    step_scores=["example_prob_diff"]
)

# Weight attribution scores by the difference in logits
out.weight_attributions("example_prob_diff")
out.show()

Attributing with saliency...: 100%|██████████| 16/16 [00:35<00:00,  2.38s/it]
/Users/barberademol/Desktop/ik-nlp-tutorials/venv/lib/python3.8/site-packages/inseq/data/attribution.py:275: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3281.)
  step_scores = self.step_scores[step_fn_id].T.unsqueeze(1)


,▁Tall,er,▁,runner,▁and,▁smaller,▁one,▁in,▁blue,▁are,▁the,▁two,▁people,.,</s>
▁,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.005,-0.0,-0.006,-0.0
premise,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.002,-0.0,-0.003,-0.0
:,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.005,-0.0,-0.006,-0.0
▁Tall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.005,-0.0,-0.011,-0.0
er,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.005,-0.0,-0.01,-0.0
▁,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.006,-0.0,-0.009,-0.0
runner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.005,-0.0,-0.014,-0.0
▁in,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.01,-0.0,-0.007,-0.0
▁orange,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.009,-0.0,-0.004,-0.0
▁and,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.023,-0.001,-0.013,-0.0
